<a href="https://colab.research.google.com/github/jonas-jun/nlp_imdb_sentiment/blob/master/sentimental_analysis_IMDb_2_200801.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torchtext
!pip install spacy
!python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [2]:
import torch
from torchtext import data
from torchtext import datasets
import random
import torch.nn as nn
import torch.optim as optim
import time # calculate time for an epoch
import spacy

## Prepare Datasets

In [12]:
SEED = 1234
torch.manual_seed(SEED)

TEXT = data.Field(tokenize='spacy', include_lengths=True) # include_lengths
LABEL = data.LabelField(dtype=torch.float)

In [13]:
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

print('number of training examples: {:,}'.format(len(train_data)))
print('number of test examples: {:,}'.format(len(test_data)))

number of training examples: 25,000
number of test examples: 25,000


In [14]:
print(vars(train_data.examples[0]))

{'text': ['THE', 'DEVIL', "'S", 'PLAYTHING', 'is', 'my', 'second', 'attempt', 'at', 'a', 'Joseph', 'Sarno', 'production', '-', 'and', 'although', 'I', 'will', 'say', 'it', 'is', 'far', 'more', 'enjoyable', 'than', 'the', 'painfully', 'dull', 'and', 'unerotic', 'Swedish', 'WILDCATS', ',', 'it', 'is', 'still', 'a', 'little', 'slow', 'and', 'un', '-', 'explicit', 'for', 'my', 'taste.<br', '/><br', '/>This', 'one', 'centers', 'around', 'a', 'group', 'of', 'vampire', 'girls', 'who', 'live', 'in', 'a', 'castle', ',', 'that', 'want', 'to', 'resurrect', 'their', 'previously', 'murdered', '"', 'leader', '"', '.', 'In', 'order', 'to', 'do', 'so', ',', 'the', 'girls', 'have', 'to', 'dance', 'around', 'naked', 'and', 'kiss', 'each', 'other', 'and', 'chant', 'weird', 'stuff', '-', 'and', 'of', 'course', 'drink', 'some', 'blood', ',', 'too', '.', 'When', 'a', 'doctor', 'and', 'her', 'brother', "'s", 'car', 'breaks', 'down', 'and', 'they', 'have', 'to', 'stop', 'at', 'the', 'castle', 'for', 'lodging'

In [15]:
train_data, valid_data = train_data.split(random_state=random.seed(SEED)) # default ratio is 0.7

print('number of training examples: {:,}'.format(len(train_data)))
print('number of validation examples: {:,}'.format(len(valid_data)))
print('number of test examples: {:,}'.format(len(test_data)))

number of training examples: 17,500
number of validation examples: 7,500
number of test examples: 25,000


In [16]:
max_vocab_size = 25000

TEXT.build_vocab(train_data, max_size=max_vocab_size, vectors='glove.6B.100d', unk_init=torch.Tensor.normal_) # meaning of arguments
LABEL.build_vocab(train_data)

In [17]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iter, valid_iter, test_iter = data.BucketIterator.splits(
    datasets=(train_data, valid_data, test_data), batch_size=BATCH_SIZE,
    sort_within_batch=True, device=device)

In [18]:
print('number of train iter: {:,}'.format(len(train_iter)))
print('number of valid iter: {:,}'.format(len(valid_iter)))
print('number of test iter: {:,}'.format(len(test_iter)))

number of train iter: 274
number of valid iter: 118
number of test iter: 391


In [19]:
int(len(train_data) / BATCH_SIZE) # approximately same with length of train iter

273

## Build the Model

- input dimension: length of one-hot vectors
- embedding dimension: size of dense word vectors, usually around 50-250 dimensions
- hidden dimension: size of the hidden states
- output dimension: number of classes, in this case 1, because only 2 cases, 0 or 1

In [21]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout=dropout)
        self.fc = nn.Linear(hidden_dim*2, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text, text_lengths):
        embedded = self.dropout(self.embedding(text))

        # pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        packed_output, (hidden, cell) = self.rnn(packed_embedded)

        # unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))

        return self.fc(hidden)

Like before, we'll create an instance of our RNN class, with the new parameters and arguments for the number of layers, bidirectionality and dropout probability.

To ensure the pre-trained vectors can be loaded into the model, the EMBEDDING_DIM must be equal to that of the pre-trained GloVe vectors loaded earlier.

We get our pad token index from the vocabulary, getting the actual string representing the pad token from the field's pad_token attribute, which is 'pad' by default.

In [22]:
input_dim = len(TEXT.vocab)
embedding_dim = 100
hidden_dim = 256
output_dim = 1
n_layers = 2
bidirectional = True
dropout = 0.5
pad_idx = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(vocab_size=input_dim,
            embedding_dim=embedding_dim,
            hidden_dim=hidden_dim,
            output_dim=output_dim,
            n_layers=n_layers,
            bidirectional=bidirectional,
            dropout=dropout,
            pad_idx=pad_idx)

In [23]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print('The model has {:,} trainable parameters'.format(count_parameters(model)))

The model has 4,810,857 trainable parameters


The final addition is copying the pre-trained word embeddings we loaded earlier into the embedding layer of out model.

We retrieve the embeddings from the field's vocab, and check they're the correct size, [vocab size, embedding dim]

In [24]:
pretrained_embeddings = TEXT.vocab.vectors
print(pretrained_embeddings.shape)

torch.Size([25002, 100])


Then, we replace the initial weights of the embedding layer with the pre-trained embeddings.

NOTE: this should always be done on the weight.data and not the weight!

In [25]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.8591, -0.7120,  0.0041,  ..., -1.1054, -0.8005,  0.2965],
        [-1.6772,  0.4427, -0.1157,  ..., -1.0376,  1.1156, -1.8099],
        [ 2.0426, -1.7188,  0.2626,  ...,  0.6916, -0.7286, -1.3932]])

As our 'unk' and 'pad' token aren't in the pre-trained vocabulary they have been initialized using unk_init (N(0,1) distribution) when building our vocab. it is preferable to initialize them both to all zeros to explicitly tell our model that, initially, they are irrelevant for determining sentiment.

We do this by manually setting their row in the embedding weights matrix to zeros. We get their row by finding the index of the tokens, which we have already done for the padding index.

NOTE: like initializing the mebeddings, this should be done one the weight.data and not the weight!

In [26]:
unk_idx = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[unk_idx] = torch.zeros(embedding_dim)
model.embedding.weight.data[pad_idx] = torch.zeros(embedding_dim)

print(model.embedding.weight.data.shape)
print(model.embedding.weight.data)

torch.Size([25002, 100])
tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.8591, -0.7120,  0.0041,  ..., -1.1054, -0.8005,  0.2965],
        [-1.6772,  0.4427, -0.1157,  ..., -1.0376,  1.1156, -1.8099],
        [ 2.0426, -1.7188,  0.2626,  ...,  0.6916, -0.7286, -1.3932]])


## Train the Model

    argument 1: parameters we'll update
    argument 2: learning rate

    > change optimization method SGD to Adam

In [27]:
optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device) # which things are should read to cuda?

/usr/local/lib/python3.6/dist-packages/torch/cuda/__init__.py:125: UserWarning: 
Tesla T4 with CUDA capability sm_75 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_70.
If you want to use the Tesla T4 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


#### Accuracy Function

In [28]:
def binary_accuracy(preds, y):
    '''
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8 NOT 8
    '''

    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() # convert to float for division
    acc = correct.sum() / len(correct)
    return acc

#### Definitions for training and evaluating

As we have set include_lengths = True, our batch.text is now a tuple with the first element being the numericalized tensor and the second element being the actual lengths of each sequence. We seperate these into their own variables, text and text_lengths, before passing them to the model.

NOTE: as we are now using dropout, we must remember to use model.train() to ensure the dropout is 'turrned on' while training.

In [29]:
def train(model, iterator, optimizer, criterion):

    epoch_loss = 0
    epoch_acc = 0
    
    model.train()

    for batch in iterator:
        
        optimizer.zero_grad()
        text, text_lengths = batch.text

        predictions = model(text, text_lengths).squeeze(1)

        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item() # .item() ?

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

As we are now using dropout, we must remember to use model.eval() to ensure the dropout is 'turned off' while evaluating

In [30]:
# do not need to optimize, do back propagation
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():
        for batch in iterator:
            text, text_lengths = batch.text

            predictions = model(text, text_lengths).squeeze(1)

            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Create function to tell us how long an apoch takes to compare training times between models

In [31]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins*60))
    return elapsed_mins, elapsed_secs

#### Train the model multiple epochs

At each epoch, if the validation loss is the best we have seen so far, we'll save the parameters of the model and then after training has finished we'll use that model on the test set

In [32]:
N_EPOCHS = 5
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss, train_acc = train(model, train_iter, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iter, criterion)
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')

    print('Epoch: {:02} | Epoch Time: {}m {}s'.format(epoch+1, epoch_mins, epoch_secs))
    print(f'\tTrain Loss: {train_loss:0.03f}, | Train_acc: {(train_acc*100):0.02f}')
    print(f'\tValid Loss: {valid_loss:0.03f}, | Valid_acc: {(valid_acc*100):0.02f}') 

Epoch: 01 | Epoch Time: 0m 37s
	Train Loss: 0.643, | Train_acc: 62.38
	Valid Loss: 0.573, | Valid_acc: 72.52
Epoch: 02 | Epoch Time: 0m 38s
	Train Loss: 0.540, | Train_acc: 73.38
	Valid Loss: 0.439, | Valid_acc: 81.24
Epoch: 03 | Epoch Time: 0m 38s
	Train Loss: 0.460, | Train_acc: 78.55
	Valid Loss: 0.363, | Valid_acc: 84.46
Epoch: 04 | Epoch Time: 0m 39s
	Train Loss: 0.454, | Train_acc: 78.83
	Valid Loss: 0.364, | Valid_acc: 84.88
Epoch: 05 | Epoch Time: 0m 39s
	Train Loss: 0.345, | Train_acc: 85.39
	Valid Loss: 0.318, | Valid_acc: 86.48


## Apply Testset

In [33]:
model.load_state_dict(torch.load('tut2-model.pt'))
test_loss, test_acc = evaluate(model, test_iter, criterion)
print('Test Loss: {:0.03f} | Test Acc: {:0.02f}%'.format(test_loss, test_acc*100))

Test Loss: 0.332 | Test Acc: 85.69%


## Test with User Input

We can now use our model to predict the sentiment of any sentence we give it. As it has benn trained on movie reviews, the sentences provided should also be movie reviews.

When using a model for inference it should always be in evaluation mode. If this tutorial is followed step-by-step then it should already be in evaluation mode (from doing evaluate on the test set), however we explicitly set it to avoid any risk

Our predict_sentiment function does a few things:

1. sets the model to evaluation mode
2. tokenizes the sentence, i.e. splits it from a raw string into a list of tokens
3. indexes the tokens by converting them into their integer representation from our vocabulary
4. gets the length of our sequence
5. converts the indexes, which are a Python list into a PyTorch tensor
6. add a batch dimension by unsqueezeing
7. converts the length into a tensor
8. squashes the output prediction from a real number between 0 and 1 with the sigmoid function
9. converts the tensor holding a single value into an integer with the item() method

We are expecting reviews with a negative sentiment to return a value close to 0 and positive reviews to return a value close to 1.

In [36]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))

    return prediction.item()

In [37]:
predict_sentiment(model, "It was really boring, and I felt that I would like to stop this travel and quit the theater.")

0.25800949335098267